<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [9]</a>'.</span>

# Installing Required Libraries & Importing Modules

In [1]:
!pip install pytesseract pdf2image pillow json5 pandas tqdm psycopg2-binary python-dotenv transformers huggingface_hub torch accelerate datasets pymongo pymupdf pdfplumber PyPDF2 pdfminer.six

  Obtaining dependency information for psycopg2-binary from https://files.pythonhosted.org/packages/61/69/3b3d7bd583c6d3cbe5100802efa5beacaacc86e37b653fc708bf3d6853b8/psycopg2_binary-2.9.10-cp311-cp311-win_amd64.whl.metadata
  Using cached psycopg2_binary-2.9.10-cp311-cp311-win_amd64.whl.metadata (5.0 kB)
  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/5f/a0/d9ef19f780f319c21ee90ecfef4431cbeeca95bec7f14071785c17b6029b/accelerate-1.10.1-py3-none-any.whl.metadata
  Using cached accelerate-1.10.1-py3-none-any.whl.metadata (19 kB)
  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/eb/62/eb8157afb21bd229c864521c1ab4fa8e9b4f1b06bafdd8c4668a7a31b5dd/datasets-4.0.0-py3-none-any.whl.metadata
  Using cached datasets-4.0.0-py3-none-any.whl.metadata (19 kB)
  Obtaining dependency information for dill<0.3.9,>=0.3.0 from https://files.pythonhosted.org/packages/c9/7a/cef76fd8438a42f96db64ddaa85280485a9c395e7df3db

ERROR: Could not install packages due to an OSError: [WinError 5] Accès refusé: 'C:\\Users\\LENOVO\\Desktop\\ENSA\\GI3\\PFE\\resume-recommender\\ai\\.venv\\Lib\\site-packages\\psycopg2\\_psycopg.cp311-win_amd64.pyd'
Check the permissions.


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Windows-compatible poppler installation
import platform
import os
import urllib.request
import zipfile
from pathlib import Path

if platform.system() == 'Windows':
    print("Setting up poppler for Windows...")
    
    # Check if poppler is already available
    poppler_path = r"C:\Program Files\poppler\bin"
    if not os.path.exists(poppler_path):
        print("Poppler not found in standard location. Checking environment...")
        # For Windows, we'll assume poppler-utils is installed or use pdf2image without it
        print("Note: If PDF processing fails, please install poppler-utils for Windows")
        print("Download from: https://github.com/oschwartz10612/poppler-windows/releases/")
    else:
        print(f"Poppler found at: {poppler_path}")
        # Add to PATH if not already there
        if poppler_path not in os.environ.get('PATH', ''):
            os.environ['PATH'] = poppler_path + ';' + os.environ.get('PATH', '')
            
print("Poppler setup complete")

Setting up poppler for Windows...
Poppler not found in standard location. Checking environment...
Note: If PDF processing fails, please install poppler-utils for Windows
Download from: https://github.com/oschwartz10612/poppler-windows/releases/
Poppler setup complete


In [3]:
!pip install -U bitsandbytes

^C


In [4]:
import os
import pytesseract
from pdf2image import convert_from_path
from PIL import Image
import re
import json5
import pandas as pd
from tqdm import tqdm
import psycopg2
from dotenv import load_dotenv
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from huggingface_hub import login
from pymongo import MongoClient
import json
import torch
import pdfplumber
import pytesseract
from pdf2image import convert_from_path
import logging

# Loading Environment Variables

In [5]:
load_dotenv()

# Hugging Face Token (optional)
hf_token = os.getenv('HF_TOKEN')
if hf_token:
    print("Hugging Face token loaded")
else:
    print("No Hugging Face token found - some features may be limited")

# MongoDB Configuration (optional)
mongo_host = os.getenv('MONGO_HOST')
mongo_port = os.getenv('MONGO_PORT')
mongo_db = os.getenv('MONGO_DB')
mongo_collection = os.getenv('MONGO_COLLECTION')

# PostgreSQL Configuration (with defaults)
postgres_host = os.getenv('POSTGRES_HOST', 'localhost')
postgres_port = os.getenv('POSTGRES_PORT', '5432')
postgres_db = os.getenv('POSTGRES_DB', 'pfe_db')
postgres_user = os.getenv('POSTGRES_USER', 'postgres')
postgres_password = os.getenv('POSTGRES_PASSWORD', '1986')

print(f"PostgreSQL config: {postgres_host}:{postgres_port}/{postgres_db}")
print(f"MongoDB config: {mongo_host}:{mongo_port}/{mongo_db} (collection: {mongo_collection})")

No Hugging Face token found - some features may be limited
PostgreSQL config: localhost:5432/pfe_db
MongoDB config: localhost:27017/PFE (collection: resumes)


# Hugging Face Connection

In [6]:
# Hugging Face Login (optional)
if hf_token:
    try:
        login(hf_token)
        print("Hugging Face login successful")
    except Exception as e:
        print(f"Hugging Face login failed: {e}")
        print("Continuing without HF authentication - some features may be limited")
else:
    print("Skipping Hugging Face login - no token provided")

Skipping Hugging Face login - no token provided


# MongoDB Connection

In [7]:
# MongoDB Connection with error handling
client = None
db = None
resumes = None

try:
    # Check if MongoDB variables are properly set
    if mongo_host and mongo_port and mongo_db and mongo_collection:
        print(f"Connecting to MongoDB at {mongo_host}:{mongo_port}")
        client = MongoClient(f"{mongo_host}:{mongo_port}")
        db = client[mongo_db]
        resumes = db[mongo_collection]
        print("MongoDB connection successful")
    else:
        print("MongoDB configuration incomplete. Skipping MongoDB connection.")
        print(f"MONGO_HOST: {mongo_host}, MONGO_PORT: {mongo_port}")
        print(f"MONGO_DB: {mongo_db}, MONGO_COLLECTION: {mongo_collection}")
except Exception as e:
    print(f"MongoDB connection failed: {e}")
    print("Continuing without MongoDB. Results will only be saved to PostgreSQL.")
    client = None
    db = None
    resumes = None

Connecting to MongoDB at localhost:27017
MongoDB connection successful


# Postgres Connection

In [8]:
def get_postgres_connection():
    try:
        conn = psycopg2.connect(
            dbname=postgres_db,
            user=postgres_user,
            password=postgres_password,
            host=postgres_host,
            port=postgres_port
        )
        print("✅ Connected to PostgreSQL database.")
        return conn
    except Exception as e:
        print(f"❌ Error connecting to PostgreSQL: {e}")
        return None

# Loading Model and Tokenizer

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [9]:
# Set the path where you want to store the model
drive_model_path = "/content/drive/MyDrive/PFE/ai_models/mixtral-8x7b"

# Check if model is already downloaded to Drive
if os.path.exists(drive_model_path):
    print("Loading model from Drive cache...")
    tokenizer = AutoTokenizer.from_pretrained(drive_model_path)
    model = AutoModelForCausalLM.from_pretrained(
        drive_model_path,
        device_map="auto"
    )
else:
    print("Downloading model from HuggingFace...")
    MODEL_NAME = "mistralai/Mixtral-8x7B-Instruct-v0.1"

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.float16
    )

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto"
    )

    # Save to Drive for future use
    print("Saving model to Google Drive for reuse...")
    tokenizer.save_pretrained(drive_model_path)
    model.save_pretrained(drive_model_path)

# Set padding token if needed
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Create pipeline
chat_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    pad_token_id=tokenizer.pad_token_id,
    return_full_text=False
)


PackageNotFoundError: No package metadata was found for bitsandbytes

# Text extraction (OCR or PyMuPDF)

In [ ]:
def extract_text(pdf_path):
    try:
        # Try PDFPlumber first (more accurate than PyMuPDF for many layouts)
        full_text = ""
        logging.getLogger("pdfminer").setLevel(logging.ERROR)
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text = page.extract_text()
                if text:
                    full_text += text + "\n"

        full_text = full_text.strip()

        # If text is empty or too short, fallback to OCR
        if not full_text or len(full_text) < 50:
            print("🔁 Switching to OCR (image-based or low-quality PDF)")
            images = convert_from_path(pdf_path)
            full_text = ""
            for i, img in enumerate(images):
                ocr_result = pytesseract.image_to_string(img)
                full_text += ocr_result + "\n"

        return full_text.strip()

    except Exception as e:
        print(f"❌ Error during text extraction: {e}")
        return None

# Clean up the text

In [ ]:
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Prompt creation


In [ ]:
def create_prompt(resume_text):
    prompt = f"""
Extract structured data from the following resume text and return a valid JSON object with the fields:

- name, age, email, phone, personal_links (github, linkedin, portfolio, indeed, others), skills, languages, experiences (job_title, company, duration, description), projects (project_title, project_summary), education (degree, institution, year), certifications, years_of_experience.

Return only a **raw JSON object**. Do NOT include any explanation, markdown, or code block (like ```json or ```python). Just the JSON like this example (no comments or extra text):
Example:
{{
  "name": "Jane Doe",
  "age": 28,
  "email": "jane.doe@example.com",
  "phone": "+212612345678",
  "personal_links": {{
    "github": "https://github.com/janedoe",
    "linkedin": "https://linkedin.com/in/janedoe",
    "portfolio": "https://janedoe.dev",
    "indeed": "https://indeed.com/profile/janedoe",
    "others": ["https://behance.net/janedoe", "https://medium.com/@janedoe"]
  }},
  "skills": ["Python", "React", "MongoDB", "Docker", "problem solving", "communication"],
  "languages": ["English", "Arabic", "French"],
  "experiences": [
    {{
      "job_title": "AI Engineer",
      "company": "TechCorp",
      "duration": "Jan 2020 - Present",
      "description": "Led a team of engineers to develop NLP models for document parsing."
    }}
  ],
  "projects": [
    {{
      "project_title": "Resume Parser",
      "project_summary": "Developed a parser using Transformers and MongoDB."
    }}
  ],
  "education": [
    {{
      "degree": "BSc in Computer Engineering",
      "institution": "ENSA",
      "year": "2022"
    }}
  ],
  "certifications": ["AWS Certified ML Engineer"],
  "years_of_experience": 5
}}

Resume Text:
{resume_text}
"""
    return prompt

# Call Hugging Face model

In [ ]:
# Call Hugging Face model
def call_AI_model(prompt):
    output = chat_pipeline(
        prompt,
        max_new_tokens=1200,
        do_sample=True,
        temperature=0.2,
        return_full_text=False
    )[0]['generated_text']
    return output

# Parse model response

In [ ]:
def parse_AI_model_output(ai_output):
    try:
        json_str = ai_output.strip().strip('```json').strip('```').strip()
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        print(f"JSON parsing error: {e}")
        return None

# Save to MongoDB

In [ ]:
def save_to_mongodb(resume):
    try:
        resumes.insert_one(resume)
        print("✅ Saved to MongoDB!")
    except Exception as e:
        print(f"❌ Error saving to MongoDB: {e}")

# Full resume processing

In [ ]:
def process_resume(pdf_path, uuid=None):
    resume_text = extract_text(pdf_path)
    if not resume_text:
        print("❌ No text found in resume. Skipping.")
        return False

    resume_text = preprocess_text(resume_text)
    print("📄 Extracted and preprocessed text:", resume_text)
    prompt = create_prompt(resume_text)
    AI_model_output = call_AI_model(prompt)
    print("ai model output:", AI_model_output)
    parsed_data = parse_AI_model_output(AI_model_output)
    print("parsed data output:", parsed_data)

    if parsed_data:
        parsed_data['uuid'] = uuid
        save_to_mongodb(parsed_data)
        print(f"✅ Successfully processed and stored resume UUID {uuid}")
        return True
    else:
        print("❌ Failed to parse resume.")
        return False

# Process all resumes from Postgres database

In [ ]:
def update_parsed_flag(resume_id):
    try:
        conn = get_postgres_connection()
        cursor = conn.cursor()
        cursor.execute("UPDATE resumes SET parsed = TRUE WHERE id = %s;", (resume_id,))
        conn.commit()
        conn.close()
        return True
    except Exception as e:
        print(f"❌ Failed to update resume ID {resume_id}: {e}")
        return False

In [ ]:
def process_all_resumes_from_db():
    conn = get_postgres_connection()
    cursor = conn.cursor()

    cursor.execute("SELECT resume_id, uuid, storage_path FROM resumes WHERE parsed = FALSE;")

    failed_resumes = []
    rows = cursor.fetchall()

    if not rows:
        print("✅ No new resumes to process.")
        return

    for row in tqdm(rows, desc="Processing Resumes"):
        resume_id, uuid, file_path = row
        print(f"\n🔍 Processing resume ID {resume_id}: {file_path}")
        try:
            success = process_resume(file_path, uuid)
            if success:
                update_parsed_flag(resume_id)
        except Exception as e:
            print(f"❌ Unexpected error while processing {file_path}: {e}")
            failed_resumes.append(file_path)

    conn.close()

    if failed_resumes:
        print("\n⚠️ Failed to process the following resumes:")
        for path in failed_resumes:
            print(f" - {path}")

# Run the processing function

In [ ]:
process_all_resumes_from_db()